In [172]:
!pip install -q selenium lxml tldextract

In [248]:
import re, os
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
from collections import Counter, defaultdict
from tqdm.auto import tqdm
from urllib.parse import urlparse
from requests import get, head
from random import sample, choice
from time import sleep
from selenium import webdriver

from xmlr import xmlparse, xmliter, XMLParsingMethods
from urllib.parse import urlparse

PATTERN_DOMAIN = '(?:http.*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*'

import tldextract
import langdetect
import json
from xml.etree import ElementTree
# import lxml

In [237]:
def extract_suffix(url):
    obj = tldextract.extract(url)
    return obj.suffix


In [238]:
# test
def test_extract_suffix(): 
    actual = extract_suffix("google.com")
    print(actual)
    actual = extract_suffix("www.google.co.th")
    print(actual)
    actual = extract_suffix("www.google.com:433")
    print(actual)
    actual = extract_suffix("https://www.google.com:433")
    print(actual)
test_extract_suffix()

com
co.th
com
com


In [252]:
def extract_domain(url):
    # t = urlparse(url)
    ext = tldextract.extract(url)
    return '.'.join(part for part in ext[:3] if part)


In [253]:
# test
def test_extract_domain(): 
    actual = extract_domain("google.com")
    print(actual)
    actual = extract_domain("www.google.com")
    print(actual)
    actual = extract_domain("www.google.com:433")
    print(actual)
    actual = extract_domain("https://www.google.com:433")
    print(actual)
test_extract_domain()

google.com
www.google.com
www.google.com
www.google.com


In [254]:
dataset = {
    "en": defaultdict(lambda: []),
    "de": defaultdict(lambda: [])
}

In [255]:
counter = Counter()
with tqdm(total=204522994) as pbar_items:
    with tqdm(total=67977) as pbar_domains:

        for d in xmliter('../data/en-de.bicleaner07.tmx', 'tu'):
#             print(d)
            
            counter['row_counter'] += 1
            en_url = d['tuv'][0]['prop']
            de_url = d['tuv'][1]['prop']

            # print("en_urls", en_url)
            # print("de_urls", de_url)

            if en_url == "unknown" or de_url == "unknown":
                counter['unkown_url_counter'] += 1
                continue

            if type(en_url) == list:
                en_url = en_url[0]
            if type(de_url) == list:
                de_url = de_url[0]

            if type(en_url) == str:
                en_domain = extract_domain(en_url)
                # if not en_domain in dataset['en'].keys():
                    # dataset['en'][en_domain] = []
                # dataset['en'][en_domain].append(en_url)
                # else:
                if len(dataset['en'][en_domain] ) == 10:
                    continue
                dataset['en'][en_domain].append(en_url)

            if type(de_url) == str:
                de_domain = extract_domain(de_url)

                # if not de_domain in dataset['de'].keys():
                #     dataset['de'][de_domain] = []
                # dataset['de'][de_domain].append(de_url)
                # else:
                if len(dataset['de'][de_domain]) == 10:
                    continue
                dataset['de'][de_domain].append(de_url)


            if len(dataset['de'].keys()) > counter['domain_counter']:
                counter['domain_counter'] += 1           
                pbar_domains.update(1)
                
            pbar_items.update(1)
            break

In [256]:
# Dump to JSON
with open('../data/en-de.domains.urls.json', 'w', encoding="utf-8") as f:
    
    json.dump(dataset, f, ensure_ascii=False)

In [199]:
list(dataset['de'].keys())[-100:]

['logosquotes.org',
 'dobner.net',
 'austrodiesel.at',
 'respect.cz',
 'clasificadoinmobiliariogratis.com',
 'knotland.org',
 'vos-mechanical.com',
 'aurel-archery.de',
 'tubatrade.hu',
 'marmotex.it',
 'geemarc.com',
 'holidays-in-komiza.com',
 'pfeifenmacher-pb.de',
 'canyoning-team.de',
 'tecma-pack.fr',
 'solfagroup.com',
 'raken.com',
 'anunciosinmobiliarioseuropa.com',
 'come-vendere-la-casa.com',
 'vendere-agli-stranieri.com',
 'vendere-casa-inglesi.com',
 'annunciimmobiliarigratisprivati.com',
 'real-estate-overseas.com',
 'annoncesimmogratuites.com',
 'inmobiliario-particulares.com',
 'thehamlet.ie',
 'warminskawinnica.pl',
 'diehl-patent.de',
 'staldijkzicht.nl',
 'freiberger-dom.de',
 'pabst-publishers.com',
 '3doptik.de',
 'durst-lackieranlagen.de',
 'conaxesstrade.com',
 'traveldata.sk',
 'daltonbv.nl',
 'amt-analysenmesstechnik.de',
 'remanenz.de',
 'charamel.com',
 'hoehne.de',
 'tinasoft.com',
 'betgun.com',
 'scienceblogs.de',
 'osteriamascaron.it',
 'isomed2000.de',
 

In [116]:
len(set(list(dataset['de'].keys())))

71097

## 2. Explore



In [74]:
def get_status(url):
  try:
    r = head(url, timeout=5)
    code = r.status_code
  except:
    code = 0  # probably bad domain name
  return code

def get_content(url):
    try:
        r = get(url)
#         print(r.text)
#         soup = BeautifulSoup(r.content, 'html.parser')
        text = r.text 
    except Exception as e:
        print(e)
        text = ""  # probably bad domain name
    
    return text


def substitute_de_to_th(de_url):
    pattern = re.compile(r'\bde\b')

    n = len(pattern.findall(de_url))
    if n == 1:

        th_url = pattern.sub("th", de_url)
        return th_url
    else:
        return None

### Find domains extension statistics

In [ ]:

urlparse(url)



### Find which domains fall in \bde\b pattern

In [75]:
# มีความหลากหลายใน url format จริงๆ สงสัยต้องไปดู code เฉพาะของเขาแล้วมั้ง น่าจะมี handcraft เยอะอยู่
# ดู en ไปแล้ว ดู mt บ้าง
PATTERN_DE = re.compile(r'\bde\b')

cnt = Counter()

res = []

to_substitute_urls = []

for domain, urls in dataset['de'].items():
    n = len(PATTERN_DE.findall(urls[0]))
    if n == 0:
        res.append(urls[0])
        to_substitute_urls((domain, urls[0]))
    cnt[n] += 1
cnt

Counter({0: 17231,
         1: 28381,
         2: 4200,
         3: 147,
         4: 19,
         6: 3,
         5: 7,
         114: 1,
         8: 4,
         9: 1,
         18: 1,
         24: 1,
         28: 1,
         33: 1,
         111: 1,
         13: 1})

In [ ]:
## Send http HEAD Request to that modified URLs ,

### 3) Substitute /de/ as /th/, call a HTTP HEAD request and check returned HTTP[link text](https:// [link text](https://)) staus


In [76]:
http_status_de = {}
counter = 0
for (domain, url) in tqdm(to_substitute_urls):
#     urls = urls[:1]
    urls_th = substitute_de_to_th(url)
#     urls_th = list(filter(lambda x: x != None, urls_th))
    
    http_status_de[domain] = { }
    http_status_de[domain]['status'] = get_status(url)
    http_status_de[domain]['th_url'] = url
    
#     if counter == 100:
#         break

    if http_status_de[domain]['status'] == 200:
        
        # Detect lang
#         print('urls_th[0]', urls_th[0])
#         content = get_content(urls_th[0])
        
#         if content == '':
#             http_status_de[domain]['lang_id'] = None
#         else:
#             print(content[:100])
#             lang = langdetect.detect(content)
#             print("Language detect:", lang)
#             http_status_de[domain]['lang_id'] = lang

        print(http_status_de[domain])

    counter += 1
    # break



{'status': [200], 'th_url': ['http://www.chaletino.com/th/Spanien/El-Gastor-6838/House-in-El-Gastor-Cadiz-102849-55851/']}
{'status': [200], 'th_url': ['https://2fish.co/th/church/does-my-church-really-matter/']}
{'status': [200], 'th_url': ['http://www.oil-painting-online.com/th/expressionism-oil-painting/li-shan-oil-painting-art.html']}
{'status': [200], 'th_url': ['http://www.bol-brac.eu/th/suchen.html?ponuda=5.10.11&mjesto=Bol']}
{'status': [200], 'th_url': ['https://www.camping.fr/th/annuaire/France/Midi-Pyrenees.html']}
{'status': [200], 'th_url': ['http://www.capetownhoteldesk.com/hotel/148076-place-on-the-bay-cape-town.th.html']}
{'status': [200], 'th_url': ['http://www.bedzzle.it/th/bed-and-breakfast/la-casa-di-ulisse-livorno-toskana/495']}
{'status': [200], 'th_url': ['http://chromebygoogle.net/th/tag/google-january/']}
{'status': [200], 'th_url': ['http://th.m-rollformingmachine.com/productimage/29413494.html']}
{'status': [200], 'th_url': ['http://www.myischia.it/th/tag/fri

In [80]:
cnt = Counter()
for item, v in http_status_de.items():
#     if v['status'][0] == 200:
    cnt[v['status'][0]] += 1

In [81]:
cnt

Counter({0: 26999,
         301: 629,
         500: 29,
         200: 179,
         302: 131,
         404: 343,
         403: 29,
         406: 23,
         521: 2,
         307: 2,
         410: 2,
         424: 1,
         409: 1,
         308: 1,
         503: 3,
         402: 1,
         303: 5,
         412: 1})

### Examples:

- https://2fish.co/th/church/does-my-church-really-matter/

- http://www.oil-painting-online.com/th/expressionism-oil-painting/li-shan-oil-painting-art.html

- https://golfauvietnam.com/orient-golf/?lang=th

- https://gsuite.google.se/intl/th/terms/sla.html

- http://www.xd-cinema.com/th/analytical-comparison-3d-4d-5d-6d-7d-9d-truck-mobile-cinema/

## Detect Language

In [83]:
def fetch_content(url):
    """fetch HTML content (excluded HTML <script> tags)"""
    pass
def detect_language(content):
    """return language id"""
    pass

In [84]:
cnt = Counter()
for item, v in http_status_de.items():
    if v['status'][0] == 200:
        url = v['th_url'][0]
        
        content = fetch_content(url)
        
        